In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os
import tqdm
import glob
import tensorflow

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import gray2rgb

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import DenseNet169 
from tensorflow.keras.applications.densenet import DenseNet121 
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout

import keras.backend as K


from typing import Optional

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,

        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

In [ ]:
train_dataset  = train_datagen.flow_from_directory(directory = '/content/drive/MyDrive/AD-CN/AD-CN/train',
                                                   target_size = (224,224),
                                                   class_mode = 'binary',
                                                   subset = 'training',
                                                   batch_size = 32)

Found 867 images belonging to 2 classes.


In [ ]:
test_dataset = test_datagen.flow_from_directory(directory = '/content/drive/MyDrive/AD-CN/AD-CN/test',
                                                  target_size = (224,224),
                                                  class_mode = 'binary',
                                                  batch_size = 32)

Found 323 images belonging to 2 classes.


In [ ]:
base_model = DenseNet121(input_shape=(224,224,3),
                   include_top=False,
                   weights="imagenet")

29084464/29084464 [==============================] - 3s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable=False


In [ ]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(64,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('selu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 5) # when i run it for 50 epochs

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
        f1_score,
]


In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=METRICS)

In [ ]:
history=model.fit(train_dataset,
                        epochs = 20,
                        verbose = 1,
                         callbacks=lr_scheduler)

Epoch 1/20
28/28 [==============================] - 609s 21s/step - loss: 0.4505 - accuracy: 0.7901 - precision: 0.7316 - recall: 0.8170 - auc: 0.8757 - f1_score: 0.7782 - lr: 0.0100
Epoch 2/20
28/28 [==============================] - 14s 504ms/step - loss: 0.3090 - accuracy: 0.8812 - precision: 0.8358 - recall: 0.9045 - auc: 0.9376 - f1_score: 0.8678 - lr: 0.0063
Epoch 3/20
28/28 [==============================] - 14s 516ms/step - loss: 0.2599 - accuracy: 0.9031 - precision: 0.8690 - recall: 0.9151 - auc: 0.9535 - f1_score: 0.8947 - lr: 0.0040
Epoch 4/20
28/28 [==============================] - 14s 496ms/step - loss: 0.2479 - accuracy: 0.9054 - precision: 0.8792 - recall: 0.9072 - auc: 0.9601 - f1_score: 0.8952 - lr: 0.0025
Epoch 5/20
28/28 [==============================] - 14s 514ms/step - loss: 0.2342 - accuracy: 0.9170 - precision: 0.8880 - recall: 0.9257 - auc: 0.9636 - f1_score: 0.9096 - lr: 0.0016
Epoch 6/20
28/28 [==============================] - 14s 496ms/step - loss: 0.2301

In [ ]:
scores = model.evaluate(test_dataset)

11/11 [==============================] - 202s 20s/step - loss: 0.2190 - accuracy: 0.9226 - precision: 0.9207 - recall: 0.9264 - auc: 0.9771 - f1_score: 0.9251


In [ ]:
print("Accuracy = ", scores[1])
print("Precision = ", scores[2])
print("Recall = ", scores[3])
print("AUC = ", scores[4])
print("F1_score = ", scores[5])

Accuracy =  0.9226006269454956
Precision =  0.9207317233085632
Recall =  0.9263803958892822
AUC =  0.9770513772964478
F1_score =  0.9250882863998413
